In [6]:
types = """
double
DScalar1<double, Eigen::Matrix<double, 6, 1>>
DScalar1<double, Eigen::Matrix<double, 8, 1>>
DScalar1<double, Eigen::Matrix<double, 12, 1>>
DScalar1<double, Eigen::Matrix<double, 18, 1>>
DScalar1<double, Eigen::Matrix<double, 24, 1>>
DScalar1<double, Eigen::Matrix<double, 30, 1>>
DScalar1<double, Eigen::Matrix<double, 60, 1>>
DScalar1<double, Eigen::Matrix<double, 81, 1>>
DScalar1<double, Eigen::Matrix<double, Eigen::Dynamic, 1, 0, 9, 1>>
DScalar1<double, Eigen::Matrix<double, Eigen::Dynamic, 1, 0, SMALL_N, 1>>
DScalar1<double, Eigen::Matrix<double, Eigen::Dynamic, 1, 0, BIG_N, 1>>
DScalar1<double, Eigen::VectorXd>
DScalar2<double, Eigen::Matrix<double, 6, 1>, Eigen::Matrix<double, 6, 6>>
DScalar2<double, Eigen::Matrix<double, 8, 1>, Eigen::Matrix<double, 8, 8>>
DScalar2<double, Eigen::Matrix<double, 12, 1>, Eigen::Matrix<double, 12, 12>>
DScalar2<double, Eigen::Matrix<double, 18, 1>, Eigen::Matrix<double, 18, 18>>
DScalar2<double, Eigen::Matrix<double, 24, 1>, Eigen::Matrix<double, 24, 24>>
DScalar2<double, Eigen::Matrix<double, 30, 1>, Eigen::Matrix<double, 30, 30>>
DScalar2<double, Eigen::Matrix<double, 60, 1>, Eigen::Matrix<double, 60, 60>>
DScalar2<double, Eigen::Matrix<double, 81, 1>, Eigen::Matrix<double, 81, 81>>
DScalar2<double, Eigen::Matrix<double, Eigen::Dynamic, 1, 0, SMALL_N, 1>, Eigen::Matrix<double, Eigen::Dynamic, Eigen::Dynamic, 0, SMALL_N, SMALL_N>>
DScalar2<double, Eigen::VectorXd, Eigen::MatrixXd>
""".strip().split("\n")

In [7]:
newline = " \\\n\t"
macro1 = f"""
#define POLYFEM_DECLARE_VIRTUAL_ELASTIC_ENERGY \\
\t{newline.join(f'virtual {t} elastic_energy(const RowVectorNd &p, const int el_id, const DefGradMatrix<{t}> &def_grad) const = 0;' for t in types)}
"""

In [8]:
macro2 = f"""
#define POLYFEM_OVERRIDE_ELASTIC_ENERGY \\
\t{newline.join(f'{t} elastic_energy(const RowVectorNd &p, const int el_id, const DefGradMatrix<{t}> &def_grad) const override {{ return elastic_energy_T<{t}>(p, el_id, def_grad); }}' for t in types)}
"""

In [9]:
macro3 = f"""
#define POLYFEM_TEMPLATE_SPECIALIZE_ELASTIC_ENERGY(NAME) \\
\t{newline.join(f'template {t} NAME::elastic_energy_T<{t}>(const RowVectorNd &p, const int el_id, const DefGradMatrix<{t}> &def_grad) const;' for t in types)}
"""

In [10]:
with open("../src/polyfem/assembler/ElasticEnergyMacros.hpp", "w") as f:
    f.write(f"""\
// This file is automatically generated by tools/assembler_macros.ipynb
#pragma once

#include <polyfem/utils/AutodiffTypes.hpp>

{macro1}\
{macro2}\
{macro3}\

namespace polyfem
{{
	template <typename T>
	using DefGradMatrix = Eigen::Matrix<T, Eigen::Dynamic, Eigen::Dynamic, 0, 3, 3>;
}}
""")